### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [5]:

data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])

ohe = OneHotEncoder()
geo_encoded = ohe.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
#save encoder data 
pickle.dump(ohe, open('ohe_ndarray.pkl', 'wb'))
#save label encoder data
pickle.dump(le, open('le.pkl', 'wb'))

# save the scaler
pickle.dump(scaler, open('scalar.pkl', 'wb'))

In [6]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model




In [7]:
# Create a KerasClassifier
model=KerasClassifier(build_fn=create_model,epochs=50,verbose=1,layers=1,neurons=32)

In [8]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [14]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)


# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# Save the best model

# After fitting
best_clf = grid_result.best_estimator_
best_model = best_clf.model_  # Note: underscore

# Save the Keras model
best_model.save('best_model.h5')

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/50


c:\personal\Churn_ANN\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\personal\Churn_ANN\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - accuracy: 0.7991 - loss: 0.4581
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.8316 - loss: 0.3988
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.8480 - loss: 0.3711
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.8539 - loss: 0.3562
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8572 - loss: 0.3495  
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - accuracy: 0.8570 - loss: 0.3455
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.8589 - loss: 0.3418
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.8579 - loss: 0.3401
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.8589 - loss: 0.3384
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - accuracy: 0.8587 - loss: 0.3379
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.8601 - loss: 0.3368
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 